In [ ]:
import numpy as np
import pandas as pd

import re
from nltk.corpus import stopwords # to remove the stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

from keras.utils import to_categorical
from keras import backend as K

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/train - finalized - train.csv')
data

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,1,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...
24856,3,3,0,0,1,My Asian friend Lily is inviting me to her bir...
24857,3,3,0,0,1,My Asian friend Lily is inviting me to her bir...
24858,3,3,0,0,1,My friend Julian won the school champian for t...
24859,3,3,0,0,1,My friend Julian won the school champian for t...


In [ ]:
data['labels'] = data['class'].map({0:"Hate Speech Detected",1:"Offensive Language Detected",2:"Neither Hate or Offensive Language Detected"})
print(data.head())

   count  hate_speech_count  offensive_language_count  neither_count  class  \
0      3                  0                         0              3      1   
1      3                  0                         3              0      1   
2      3                  0                         3              0      1   
3      3                  0                         2              1      1   
4      6                  0                         6              0      1   

                                               tweet  \
0  !!! RT @mayasolovely: As a woman you shouldn't...   
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

                        labels  
0  Offensive Language Detected  
1  Offensive Language Detected  
2  Offensive Language Detected  
3  Offensive Language Detected  
4  Offensive Language D

In [ ]:
# dataset shape to know how many tweets in the datasets
print(f"num of tweets: {data.shape}")

# extract the text and labels
tweet = list(data['tweet'])
labels = list(data['labels'])

num of tweets: (24861, 7)


In [ ]:
#notes : all of the function taking 1 text at a time
stop_words = set(stopwords.words('english'))
# add rt to remove retweet in dataset (noise)
stop_words.add("rt")

# remove html entity:
def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

In [ ]:
# change the user tags
def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)

    return text

# remove urls
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

In [ ]:
def remove_noise_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')

    return text
  # remove stopwords
def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text

In [ ]:
def preprocess(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_entity(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove trailing stuff
    clean = [remove_noise_symbols(text) for text in clean]
    # remove stopwords
    clean = [remove_stopwords(text) for text in clean]

    return clean

In [ ]:
# call the cleaning function
data["tweet"] = preprocess(tweet)

In [ ]:
x = np.array(data["tweet"])
y = np.array(data["labels"])
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x = cv.fit_transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
test_data = "My Asian friend Lily is inviting me to her birthday dinner tonight. I'm so excited! I've never been to an Asian household before. It must be filled with oriental flowers that I've never seen before. I wonder what we will have for dinner. I'm a big fan of rice. oh my god maybe she will prepare dog meat for us too. I've never tried dogs meat before so I don't know how to feel about that"
df = cv.transform([test_data]).toarray()
print(model.predict(df))

['Offensive Language Detected']


In [ ]:
# import pickle
# pickle.dump(model,open("model.pkl","wb"))

In [ ]:
# model1 = pickle.load(open('/content/drive/MyDrive/model.pkl',"rb"))
test_data="My Asian friend Lily is inviting me to her birthday dinner tonight. I'm so excited! I've never been to an Asian household before. It must be filled with oriental flowers that I've never seen before. I wonder what we will have for dinner. I'm a big fan of rice. oh my god maybe she will prepare dog meat for us too. I've never tried dogs meat before so I don't know how to feel about that"
df = cv.transform([test_data]).toarray()
print(model.predict(df))

['Offensive Language Detected']


In [ ]:
!pip3 install flask_ngrok
!pip3 install pyngrok==4.1.1
!ngrok authtoken '2bFHzyDrvYy1BPHTyJ7m42eA2tZ_4uxmxZRaYHazrbXVvsYP9'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [146]:
from flask_ngrok import run_with_ngrok
import numpy as np
from flask import Flask, request, render_template
import pickle

template_folder = '/content/drive/MyDrive/Colab Notebooks/UTSHackathon/template'
static_folder = '/content/drive/MyDrive/Colab Notebooks/UTSHackathon/static'

app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)
run_with_ngrok(app)

model1 = pickle.load(open('/content/drive/MyDrive/model.pkl',"rb"))

@app.route('/aboutUS', methods=['GET'])
def aboutUS():
  return render_template("aboutus.html")

@app.route('/', methods=['GET'])
def home():
  return render_template("index (1).html")

@app.route('/login', methods=['GET'])
def login():
  return render_template("login.html")

@app.route('/yourvoice', methods=['GET'])
def yourvoice():
  return render_template("yourvoice.html")

@app.route('/Send', methods=['POST','GET'])
def predict():
  test_data= request.form.get('message')
  df = cv.transform([test_data]).toarray()
  predictions = model.predict(df)
  print(predictions)
  if predictions == ['Offensive Language Detected']:
    return render_template("index (1).html", pred="{}.Your passage may contain sterotypes or aggresive language that are offensive towards a certain minority group.\nIt is always important to consider the context you are speaking in and respect the culture and beliefs of other groups. Please consider revising your sentence.".format(predictions))
  elif predictions == ['Hate Speech Detected']:
    return render_template("index (1).html", pred="{}. Your passage contains hate language towards someone or some group.\nNote that such unfriendly language may greatly jeopardize their mental well=being and reflect badly on you\nWe understand that you might have written it in a frustrating context, but it is always crucial to consider the impacts of your words on others. Please do not swear at or target others.".format(predictions))
  else:
    return render_template("index (1).html", pred="{}. Your passage is good! Good job helping us break the societal barriers of biases and stereotypes!".format(predictions))


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a7dc-35-194-189-59.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:50] "GET /static/style%20(1).css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:50] "GET /static/DEI.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:53] "GET /yourvoice HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 16:27:53] "GET /style.css HTTP/1.1" 404 -
